In [1]:
from __future__ import print_function
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
import os

import numpy as np

from torch_model import TextRNN, TextCNN
from cnews_loader import read_vocab, read_category, batch_iter, process_file, build_vocab

2023-12-13 15:51:00.651436: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 15:51:00.811854: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-13 15:51:00.815610: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr

In [2]:
base_dir = 'cnews'
train_dir = os.path.join(base_dir, 'train.txt')
test_dir = os.path.join(base_dir, 'test.txt')
val_dir = os.path.join(base_dir, 'dev.txt')
vocab_dir = os.path.join(base_dir, 'vocab.txt')

pad_len = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def evaluate(model, Loss, x_val, y_val):
    """测试集上准确率评估"""
    batch_val = batch_iter(x_val, y_val, 64)
    acc = 0
    los = 0
    for x_batch, y_batch in batch_val:
        size = len(x_batch)
        x = np.array(x_batch)
        y = np.array(y_batch)
        x = torch.LongTensor(x).to(device)
        y = torch.Tensor(y).to(device)
        # y = torch.LongTensor(y)
        # x = Variable(x)
        # y = Variable(y)
        out = model(x)
        loss = Loss(out, y)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        loss_value = np.mean(loss.cpu().detach().numpy())
        accracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
        acc +=accracy*size
        los +=loss_value*size
    return los/len(x_val), acc/len(x_val)

In [7]:
def train():
    x_train, y_train = process_file(train_dir, word_to_id, cat_to_id,pad_len)#获取训练数据每个字的id和对应标签的oe-hot形式
    x_val, y_val = process_file(val_dir, word_to_id, cat_to_id,pad_len)
    #使用LSTM或者CNN
    model = TextRNN()
    
    model.to(device)
    # model = TextCNN()
    
    #选择损失函数
    Loss = nn.MultiLabelSoftMarginLoss()
    # Loss = nn.BCELoss()
    # Loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(),lr=0.001)
    best_val_acc = 0
    for epoch in range(10):
        i = 0
        print('epoch:{}'.format(epoch))
        batch_train = batch_iter(x_train, y_train,64)
        for x_batch, y_batch in batch_train:
            i +=1
            # print(i)
            x = np.array(x_batch)
            y = np.array(y_batch)
            x = torch.LongTensor(x).to(device)
            y = torch.Tensor(y).to(device)
            # y = torch.LongTensor(y)
            # x = Variable(x)
            # y = Variable(y)
            out = model(x)
            loss = Loss(out,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 对模型进行验证
            if i % 300 == 0:
                los, accracy = evaluate(model, Loss, x_val, y_val)
                print('loss:{},accracy:{}'.format(los, accracy))
                if accracy > best_val_acc:
                    torch.save(model.state_dict(), 'model_params.pkl')
                    best_val_acc = accracy

In [8]:
#获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
#获取训练文本中所有出现过的字及其所对应的id
build_vocab(train_dir, vocab_dir)
words, word_to_id = read_vocab(vocab_dir)
#获取字数
vocab_size = len(words)
print('train')
train()

train
epoch:0


/home/ma-user/anaconda3/envs/PyTorch-1.4/lib/python3.7/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss:0.7110180891036987,accracy:0.4292
loss:0.7002470533370971,accracy:0.5507
loss:0.6924094657897949,accracy:0.6278
loss:0.6883277956008911,accracy:0.6716
loss:0.6867353130340577,accracy:0.6846
loss:0.684760183429718,accracy:0.7039
loss:0.6820969957351685,accracy:0.7305
loss:0.6805212617874146,accracy:0.7462
loss:0.6795818076133728,accracy:0.7574
epoch:1
loss:0.6779771144866943,accracy:0.7727
loss:0.6792295835494995,accracy:0.7596
loss:0.6767408140182495,accracy:0.7852
loss:0.6763853832244873,accracy:0.7868
loss:0.6766004669189453,accracy:0.7867
loss:0.675402562904358,accracy:0.7966
loss:0.6748628734588623,accracy:0.8037
loss:0.6745181085586548,accracy:0.8072
loss:0.674998116493225,accracy:0.802
epoch:2
loss:0.6739078409194946,accracy:0.8136
loss:0.673659796333313,accracy:0.8142
loss:0.6731851741790772,accracy:0.8193
loss:0.6729810678482055,accracy:0.8229
loss:0.6738767807006836,accracy:0.8117
loss:0.6731948491096497,accracy:0.82
loss:0.6727128234863281,accracy:0.8241
loss:0.672724302